In [1]:
import requests as r
import pandas as pd
# from decimal import Decimal

import sys
# sys.path.append("../helper_functions")
# import duneapi_utils as d
# import pandas_utils as p
# sys.path.pop()

import time
import os
import dotenv
from google.cloud import bigquery
from google.oauth2 import service_account

dotenv.load_dotenv()

True

In [2]:
dataset_id = 'api_table_uploads' #schema name
# table_upload_paths = [

# ]
file_to_upload = '../op_chains_tracking/outputs/chain_data/ch_template_alltime_chain_activity' + '.csv'

In [3]:
# Check if running locally
is_running_local = os.environ.get("IS_RUNNING_LOCAL", "False").lower() == "true"
print(is_running_local)

True


In [4]:
# Set the environment variable to the path of your service account key file
if is_running_local: #GH Action was weird with this, so forcing the datatype here
        print("Running locally")
        # Path to your local service account key file
        service_account_key_path = os.getenv("PATH_TO_BQ_CREDS")
        credentials = service_account.Credentials.from_service_account_file(service_account_key_path)
        # pandas_gbq.context.credentials = service_account.Credentials.from_service_account_file(os.getenv("PATH_TO_BQ_CREDS"))
else: #Can't get the Github Action version to work
        print('not running local')
        # Set the Google Cloud service account key from GitHub secret
        service_account_key = os.getenv("BQ_APPLICATION_CREDENTIALS")
        credentials = service_account.Credentials.from_service_account_info(service_account_key)
        # Set the environment variable to the path of your service account key file
        # os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_key
project_id = os.getenv("BQ_PROJECT_ID")

client = bigquery.Client(credentials=credentials, project=project_id)

Running locally


In [5]:
# Create a job configuration to overwrite the table
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)


In [8]:
# Load DF
df = pd.read_csv(file_to_upload)
# Extract the table name from the file path
table_id = os.path.splitext(os.path.basename(file_to_upload))[0]
# display(df)

ch_template_alltime_chain_activity


In [9]:
# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(
    df, f"{dataset_id}.{table_id}", job_config=job_config
)

NotFound: 404 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/oplabs-tools-data/jobs?uploadType=multipart: Not found: Dataset oplabs-tools-data:api_table_uploads

In [ ]:
# Wait for the job to complete
job.result()

print(f"Data loaded successfully to {dataset_id}.{table_id}")